In [7]:
import os
import pandas as pd
import re
import matplotlib.pyplot as plt

# Set the folder path containing the CSV files
folder_path = "../data/raw"  # Change this if your files are in a different directory


# Loop through files in the directory
for filename in os.listdir(folder_path):
    match = re.match(r"gpa(\d{4})\.csv", filename)
    if match:
        year = match.group(1)
        var_name = f"gpa{year}"
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        globals()[var_name] = df
        
gpa_vars = [var for var in globals() if var.startswith("gpa") and isinstance(globals()[var], pd.DataFrame) and var != "gpa_master"]



In [8]:
gpa_master = pd.concat([globals()[var] for var in gpa_vars], ignore_index=True)
gpa_master = gpa_master.map(lambda x: x[2:-1] if isinstance(x, str) and x.startswith('b') else x)

In [9]:


# Combine the columns, prioritizing non-null values in 'bound_for'
gpa_master['bound_for_combined'] = gpa_master['bound_for'].combine_first(gpa_master['BOUND_FOR'])

# Drop the original columns
gpa_master.drop(columns=['bound_for', 'BOUND_FOR'], inplace=True)

# Rename to a standard name (optional)
gpa_master.rename(columns={'bound_for_combined': 'bound_for'}, inplace=True)


gpa_master.drop(columns=['DIPLOMA_MET','UNWEIGHTED_RANK_DATE','WEIGHTED_RANK_DATE','diploma_type','DIPLOMA_TYPE'], inplace=True)

from dateutil.parser import parse

def fast_parse_dates(series):
    # Try general parse first (fastest)
    parsed = pd.to_datetime(series, errors='coerce')

    # Optionally: fallback to common format if still missing
    fallback = pd.to_datetime(series, format='%m/%d/%Y', errors='coerce')
    parsed = parsed.fillna(fallback)

    return parsed.dt.year


gpa_master['entry_year'] = fast_parse_dates(gpa_master['NINTHGRADEENTRY'])
gpa_master['grad_year'] = fast_parse_dates(gpa_master['DIPLOMA_ISSUED'])


gpa_master.drop(columns=['NINTHGRADEENTRY','DIPLOMA_ISSUED'],inplace=True)


In [10]:

gpa_master.dropna(inplace=True)


In [11]:
gpa_master = gpa_master[(gpa_master['gpa_unweighted'] <= 4) & (gpa_master['gpa_weighted'] <= 6)]
gpa_master = gpa_master[(gpa_master['entry_year'] < 2030)]